In [111]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader, Subset

import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier

In [112]:
#feature extraction indexing for classes
def select_class_index(labels, n):
    labels = np.array(labels)
    selected = []

    for c in range(10):
        idx = np.where(labels == c)[0][:n]
        selected.extend(idx)

    return selected

In [107]:
# load in cifar-10 dataset
train_set = torchvision.datasets.CIFAR10(root="F:/CIFAR10_Project/data", train=True, download=True)
test_set = torchvision.datasets.CIFAR10(root="F:/CIFAR10_Project/data", train=False, download=True)

# select the first 500 train images per class
train_indices = select_class_index(train_set.targets, 500)
train_subset = torch.utils.data.Subset(train_set, train_indices)

# select the first 100 test images
test_indices = select_class_index(test_set.targets, 100)
test_subset = torch.utils.data.Subset(test_set, test_indices)

print(len(train_subset))  # Should be 500 * 10 = 5000
print(len(test_subset))   # Should be 1000

print(len(train_indices))  # 5000
print(set([train_set.targets[i] for i in train_indices])) # check how many classes ( should be 0-9 )

5000
1000
5000
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [108]:
#resize to 224x224x3
transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

In [82]:
# use dataloaders to hold the resized images

train_subset.dataset.transform = transform_resnet
test_subset.dataset.transform = transform_resnet

train_loader = DataLoader(train_subset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

In [83]:
# load RestNet-18 and remove the last layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = ResNet18_Weights.DEFAULT
resnet = resnet18(weights=weights)
resnet.fc = nn.Identity()
resnet = resnet.to(device)
resnet.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [90]:
# resnet-18 feature extractor
#def extract_features(dataset):
   # loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)
def extract_features(dataset, batch_size=64):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    feats = []
    labels = []

    with torch.no_grad():
        for images, lbls in loader:
            images = images.to(device)
            f = resnet(images)
            feats.append(f.cpu())
            labels.append(lbls)

    return torch.cat(feats), torch.cat(labels)

In [91]:
#train_features, train_labels = extract_features(train_subset.dataset)
#test_features, test_labels = extract_features(test_subset.dataset)
train_features, train_labels = extract_features(train_subset)
test_features, test_labels = extract_features(test_subset)

In [92]:
# TypeError: can't convert cuda:0 device type tensor to numpy. 
# Use Tensor.cpu() to copy the tensor to host memory first.

train_features_np = train_features.cpu().numpy()
test_features_np = test_features.cpu().numpy()

pca = PCA(n_components=50)
train_pca = pca.fit_transform(train_features_np)
test_pca  = pca.transform(test_features_np)

print(train_pca.shape, test_pca.shape)

(5000, 50) (1000, 50)


In [120]:
class BasicDecisionTree:
    def __init__(self, max_depth=50, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        prob = counts / counts.sum()
        return 1 - np.sum(prob ** 2)

    def split(self, X_column, threshold):
        left_index = np.where(X_column <= threshold)[0]
        right_index = np.where(X_column > threshold)[0]
        return left_index, right_index

    def best_split(self, X, y):
        m, n = X.shape
        if m <= 1:
            return None, None

        parent_gini = self.gini(y)
        best_gini = 1
        best_index, best_thr = None, None

        for feature_index in range(n):
            thresholds = np.unique(X[:, feature_index])
            for thr in thresholds:
                left_index, right_index = self.split(X[:, feature_index], thr)
                if len(left_index) == 0 or len(right_index) == 0:
                    continue

                gini_left = self.gini(y[left_index])
                gini_right = self.gini(y[right_index])
                weighted_gini = (len(left_index) * gini_left + len(right_index) * gini_right) / m

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_index = feature_index
                    best_thr = thr

        print(f"Best split: feature {best_index}, threshold {best_thr}, gini {best_gini}")
        if best_gini >= parent_gini:
            return None, None

        return best_index, best_thr

    def build_tree(self, X, y, depth=0):
        #num_samples_per_class = [np.sum(y == c) for c in np.unique(y)]
        #predicted_class = np.argmax(num_samples_per_class)
        unique_classes = np.unique(y)
        counts = [np.sum(y == c) for c in unique_classes]
        predicted_class = unique_classes[np.argmax(counts)]

        node = {
            'depth': depth,
            'num_samples': len(y),
            'predicted_class': predicted_class
        }

        if depth < self.max_depth and len(y) >= self.min_samples_split:
            feature_index, threshold = self.best_split(X, y)
            if feature_index is not None:
                print(f"Depth {depth}: trying to split, samples: {len(y)}")
                left_index, right_index = self.split(X[:, feature_index], threshold)
                node['feature_index'] = feature_index
                node['threshold'] = threshold
                node['left'] = self.build_tree(X[left_index], y[left_index], depth + 1)
                node['right'] = self.build_tree(X[right_index], y[right_index], depth + 1)

                

        return node

    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    def predict_sample(self, node, x):
        if 'feature_index' not in node:
            return node['predicted_class']

        if x[node['feature_index']] <= node['threshold']:
            return self.predict_sample(node['left'], x)
        else:
            return self.predict_sample(node['right'], x)

    def predict(self, X):
        return np.array([self.predict_sample(self.tree, x) for x in X])

In [121]:
# training the model
tree = BasicDecisionTree(max_depth=50)
tree.fit(train_pca, train_labels.numpy())

Best split: feature 0, threshold 1.9783527851104736, gini 0.8245408416218657
Depth 0: trying to split, samples: 5000
Best split: feature 2, threshold 1.516326904296875, gini 0.8037180180908949
Depth 1: trying to split, samples: 3188
Best split: feature 4, threshold 0.9772797226905823, gini 0.766956098647038
Depth 2: trying to split, samples: 2132
Best split: feature 1, threshold -0.5513801574707031, gini 0.7431004757435223
Depth 3: trying to split, samples: 1382
Best split: feature 3, threshold -1.8210580348968506, gini 0.7665291482094083
Depth 4: trying to split, samples: 394
Best split: feature 4, threshold -0.0821700170636177, gini 0.7327475993354172
Depth 5: trying to split, samples: 178
Best split: feature 2, threshold -2.665848731994629, gini 0.7656949247094824
Depth 6: trying to split, samples: 133
Best split: feature 5, threshold -2.712825059890747, gini 0.38567639257294406
Depth 7: trying to split, samples: 39
Best split: feature 4, threshold -3.074258804321289, gini 0.0
Depth

In [122]:
# test on test samples
tree_prediction = tree.predict(test_pca)

In [123]:
print("Feature variance per column (first 10):", np.var(train_pca, axis=0)[:10])

Feature variance per column (first 10): [28.053894  12.687464  11.437771  10.900621   7.165168   6.982128
  6.057063   5.4317107  5.1885505  4.4142537]


In [124]:
print("train_pca shape:", train_pca.shape)
print("train_labels shape:", train_labels.shape)
print("Unique train labels:", np.unique(train_labels.numpy()))

train_pca shape: (5000, 50)
train_labels shape: torch.Size([5000])
Unique train labels: [0 1 2 3 4 5 6 7 8 9]


In [129]:
# Scikit's decision tree

dt_sklearn = DecisionTreeClassifier(max_depth=50, random_state=42)

X_train = train_pca
y_train = train_labels.numpy()
X_test = test_pca
y_test = test_labels.numpy()

# Train using features 
dt_sklearn.fit(train_pca, y_train)

# Predict on test features
y_pred_sklearn = dt_sklearn.predict(test_pca)

In [131]:
# save the model

import pickle

with open('basic_decision_tree.pkl', 'wb') as f:
    pickle.dump(tree, f)

In [ ]:
# to load the model

# Load the model from the file
#with open('basic_decision_tree.pkl', 'rb') as f:
    #loaded_tree = pickle.load(f)

# Use loaded_tree to predict
#y_pred = loaded_tree.predict(test_pca)

In [130]:
# i've already made an evaluation script from scratch for Naive bayes.
# I will now use the integrated evaluation functions from sklearn

def evaluate_model(y_true, y_pred):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='macro'))
    print("Recall :", recall_score(y_true, y_pred, average='macro'))
    print("F1-score :", f1_score(y_true, y_pred, average='macro'))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

print("Evaluation of Basic Decision Tree:")
evaluate_model(test_labels.numpy(), tree_prediction)

print("\nEvaluation of Scikit's Decision Tree:")
evaluate_model(y_test, y_pred_sklearn)

Evaluation of Basic Decision Tree:
Accuracy: 0.606
Precision: 0.6055026088101922
Recall : 0.6060000000000001
F1-score : 0.6049160362663388
Confusion Matrix:
 [[53  3 10  1  1  2  3  0 16 11]
 [ 6 74  2  0  0  0  0  0  6 12]
 [ 5  1 44  7  7  9 18  5  4  0]
 [ 2  1  7 52  2 18 11  5  0  2]
 [ 5  0 10  6 56  4  4 14  0  1]
 [ 0  0 10 15  6 56  6  6  0  1]
 [ 3  0  5  8  4  4 72  2  2  0]
 [ 1  0  3  7 17 10  1 59  1  1]
 [16  5  1  1  0  0  0  2 67  8]
 [ 4  9  1  3  0  1  0  1  8 73]]

Evaluation of Scikit's Decision Tree:
Accuracy: 0.613
Precision: 0.6108245727916166
Recall : 0.613
F1-score : 0.6113921468162594
Confusion Matrix:
 [[56  5  5  3  3  2  1  1 18  6]
 [ 4 75  1  1  0  1  0  0  5 13]
 [ 8  0 42 10  9  7 15  8  1  0]
 [ 1  1  9 48  4 19  8  8  1  1]
 [ 3  1  9  4 58  4  6 15  0  0]
 [ 1  1 11 15  3 56  8  5  0  0]
 [ 2  0  7  6  7  3 72  2  1  0]
 [ 4  1  6  4 11  9  1 60  4  0]
 [19  3  1  1  0  0  1  1 68  6]
 [ 4  9  0  1  0  1  0  1  6 78]]
